In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
import time
import pandas as pd
import scipy.stats as stats
import scipy.sparse as sparse
from tqdm import tqdm

In [5]:
from google.colab import files
uploaded = files.upload()

Saving w500 to w500


In [2]:
#Global variables
n=500
R = 1
pz=0.5  #prob of element being set to zero

#Function that generates testcases like makedata.m
def generateInput():
    rvs = stats.norm().rvs
    X = sparse.random(n, n, density=pz, data_rvs=rvs)
    upper_X = sparse.triu(X) 
    result = upper_X + upper_X.T - sparse.diags(X.diagonal()) - sparse.diags(X.diagonal())
    result = result.toarray()
    result = np.array(result) #Frustrated result
    for i in range(n):
      for j in range(n):
        result[i,j] = (result[i,j]>0).astype('int') - (result[i,j]<0).astype('int')
    ferro = np.abs(result) #Ferromagnetic result
    return result, ferro

#Function that calculates the energy of a system
def calc_energy(x,w): 
  return -1*np.matmul(np.matmul(w,x),x.T)/2

#Function that performs a flip
def flipR(x,E):
  result = np.copy(x)
  y = random.randint(0,len(x)-1)

  #Flipping
  result[y]*=-1

  #Calculating new energy
  E2 = E-2*np.matmul(w[y],result)*result[y]

  return result, E2

In [24]:
#Initialize a suitable temperature by attempting 1000 flips and choosing the max energy difference as initial T
def initT(x,w):
  t_array = []
  E = calc_energy(x,w)
  T = 1000

  #Run 1000 iterations to try
  for i in range(1000):
    x2, E2 = flipR(x,E)  #Perform flip
    a = np.exp(-(E2-E)/T)

    #Appending difference if accepted
    if a > 1:
        t_array.append(E2-E)
    
    elif random.random() < a:
        t_array.append(E2-E)
    
  return np.max(t_array) #Return maximum of differences

#Perform iterative improvement method
def iterativeImprovementTest(w):
  max_patience = 1000 #If the energy does not decrease 2000 tries in a row the iteration stops
  patience = 0
  x = np.random.choice([-1,1],n) 
  E = calc_energy(x,w)

  while patience < max_patience:
    x2, E2 = flipR(x,E) #Perform flip

    #If energy smaller, make this new x
    if E2 < E:
      x = np.copy(x2)
      E = E2
      patience = 0
    else:
      patience += 1
    
  return E

#Perform k restarts 
def iterativeImprovementRestarts(k,w):
  t = time.time() #Take time
  energies = []

  #K restarts
  for i in range(k):
    energies.append(iterativeImprovementTest(w))

  return np.min(energies), time.time()-t #Return smallest energy after k restarts, and time for one iteration

#Run N times and take mean and std of N tries
def runNIterative(N,k,w):
  energies = []
  times = []

  #Run N times
  for i in tqdm(range(N)):
    e,t = iterativeImprovementRestarts(k,w)
    energies.append(e) #Append energies
    times.append(t) #Append times

  return np.mean(energies), np.std(energies), np.mean(times)

#Perform exponential schedule
def expSchedule(f,L,w):
  #Variables
  t = time.time()
  x = np.random.choice([-1,1],n)
  E = calc_energy(x,w)
  T = initT(x,w)
  beta = 1/T 
  stdE = 2
  mean_energies = []
  std_energies = []

  #While loop that runs until the std of E is 0 between loops
  while stdE > 0:
    energies = []

    #Looping L times
    for i in range(L):
      x2, E2 = flipR(x,E) #Perform flip

      a = np.exp(-(E2-E)/T) #Calculating a for MH

      #MH acceptance
      if a > 1:
        x = np.copy(x2)
        E = E2
      elif random.random() < a:
        x = np.copy(x2)
        E = E2

      energies.append(E)

    #Storing values
    stdE = np.std(energies)
    
    #Iterating exponential schedule
    beta = f*beta
    T = 1/beta
     
  return energies[-1], time.time()-t

#Perform Aarts and Korst schedule
def akSchedule(dbeta,L,w):
  #Variables
  t = time.time()
  x = np.random.choice([-1,1],n)
  E = calc_energy(x,w)
  T = initT(x,w)
  beta = 1/T 
  stdE = 2
  mean_energies = []
  std_energies = []
  temperatures = []
  counter = 0

  #Looping until std = 0 between runs
  while stdE > 0:
    energies = []
    counter += 1

    #Looping L times
    for i in range(L):
      x2, E2 = flipR(x,E) #Perform flip
      a = np.exp(-(E2-E)/T) #Calculating a for MH

      #MH acceptance
      if a > 1:
        x = np.copy(x2)
        E = E2
      elif random.random() < a:
        x = np.copy(x2)
        E = E2

      energies.append(E)
    
    #Storing values
    mean_energies.append(np.mean(energies))
    stdE = np.std(energies)
    std_energies.append(stdE)
    temperatures.append(beta)

    #Iterating AK schedule
    if stdE > 0:
      beta = beta + dbeta/np.sqrt(stdE)
      T = 1/beta

  #Plotting to show for one cycle
  #plt.plot(range(counter),mean_energies)
  #plt.xlabel("k")
  #plt.ylabel("mean E")  
  #plt.ylim((-8000,0))
  #plt.show()

  #plt.plot(range(counter),std_energies)
  #plt.xlabel("k")
  #plt.ylabel("std E")
  #plt.ylim((0,600))
  #plt.show()

  #plt.plot(range(counter),temperatures)
  #plt.xlabel("k")
  #plt.ylabel(r'$ \beta $')
  #plt.ylim((0,0.5))
  #plt.show()

  #print(counter)

  return energies[-1], time.time()-t

#Run a schedule N times and get mean energy from N runs
def runNAnnealing(N,fdbeta, L, w):
  times = []
  energies = []

  for i in tqdm(range(N)):
    #Change for different schedules
    E, t = expSchedule(fdbeta,L,w)
    #E, t = akSchedule(fdbeta,L,w)

    times.append(t)
    energies.append(E)

  #Calculating means
  mean_time = np.mean(times)
  mean_energies = np.mean(energies)
  std_energies = np.std(energies)

  return mean_energies, std_energies, mean_time

  


In [28]:
#a, b = generateInput()
#w = a
df = pd.read_csv("w500", header=None, delim_whitespace=True)
w = df.to_numpy()
N = 20
#kvals = [20,100,200,500,1000,2000,4000]
kvals = [4000]
#kvals = range(1,26)
#dbetas = [0.1,0.01,0.001]
#dbetas = [0.001]
#fs = [1.01, 1.001, 1.0002]
fs = [1.0002]
means = []
stds = []
times = []

for f in fs:
  #m, s, t = runNIterative(N,k,w)
  m, s, t = runNAnnealing(N, f, 1000, w)
  means.append(m)
  stds.append(s)
  times.append(t)

print(means)
print(stds)
print(times)

#Plotting for exercise 2a)
#plt.errorbar(kvals,means,yerr=stds,fmt='o')
#plt.title("Mean Energy over 20 runs")
#plt.xlabel("K")
#plt.ylabel("Mean Energy")
#plt.show()



100%|██████████| 20/20 [1:26:50<00:00, 260.52s/it]

[-6583.3]
[28.20833210241258]
[260.51436017751695]
